# Incident Replay Notebook

1. `ops/incidents/<incident_id>/incident.json` を選択
2. 対象期間の CSV を整え、manifest 指定で `scripts/run_sim.py --manifest ...` を再実行
3. 結果や考察を `replay_notes.md` に反映


In [1]:
from pathlib import Path
import json

# === 設定を更新してください ===
INCIDENT_ID = "20250101-0900_USDJPY_drawdown"  # incident.json を置いたディレクトリ名
PROJECT_ROOT = Path.cwd().resolve()
if not (PROJECT_ROOT / "scripts").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent
INCIDENT_DIR = PROJECT_ROOT / "ops" / "incidents" / INCIDENT_ID
INCIDENT_PATH = INCIDENT_DIR / "incident.json"
assert INCIDENT_PATH.exists(), f"incident not found: {INCIDENT_PATH}"
incident = json.loads(INCIDENT_PATH.read_text(encoding="utf-8"))
incident


{'incident_id': '20250101-0900_USDJPY_drawdown',
 'symbol': 'USDJPY',
 'mode': 'conservative',
 'start_ts': '2025-10-02T15:35:00Z',
 'end_ts': '2025-10-02T19:20:00Z',
 'trigger': 'max_drawdown',
 'severity': 'high',
 'loss_pips': -37.9,
 'snapshot_state': 'ops/state_archive/day_orb_5m/USDJPY/conservative/20250924_121238_state.json',
 'notes': 'USDJPY sold off from the post-London high into the US afternoon, breaking the 147.30 support pocket and triggering the internal max drawdown alert after a 37.9 pip slide.',
 'tags': ['tokyo', 'drawdown', 'liquidity']}

In [ ]:
import shutil
import subprocess
import sys
import json
from pathlib import Path
from pprint import pprint

# === run_sim.py を manifest ベースで再実行 ===
symbol = incident["symbol"]
mode = incident.get("mode", "conservative")
start_ts = incident["start_ts"]
end_ts = incident["end_ts"]

csv_path = PROJECT_ROOT / "validated" / symbol / "5m.csv"
assert csv_path.exists(), f"missing bars CSV: {csv_path}"

out_dir = PROJECT_ROOT / "runs" / f"incident_{INCIDENT_ID}"
out_dir.mkdir(parents=True, exist_ok=True)
json_out = out_dir / "metrics.json"

prepared_csv = csv_path
with csv_path.open(encoding="utf-8") as src:
    first_line = src.readline()
    needs_header = "timestamp" not in first_line.lower()
if needs_header:
    prepared_csv = out_dir / "source_with_header.csv"
    if not prepared_csv.exists():
        with csv_path.open(encoding="utf-8") as src, prepared_csv.open("w", encoding="utf-8", newline="") as dst:
            dst.write("timestamp,symbol,tf,o,h,l,c,v,spread
")
            shutil.copyfileobj(src, dst)

manifest_value = incident.get("manifest", "configs/strategies/day_orb_5m.yaml")
manifest_path = Path(manifest_value)
if not manifest_path.is_absolute():
    manifest_path = PROJECT_ROOT / manifest_path
assert manifest_path.exists(), f"manifest not found: {manifest_path}"

cmd = [
    sys.executable,
    str(PROJECT_ROOT / "scripts" / "run_sim.py"),
    "--manifest", str(manifest_path),
    "--csv", str(prepared_csv),
    "--start-ts", start_ts,
    "--end-ts", end_ts,
    "--equity", str(incident.get("equity", 100000)),
    "--json-out", str(json_out),
    "--out-dir", str(out_dir),
]

print("Running:", " ".join(cmd))
run = subprocess.run(cmd, check=False)
print("returncode=", run.returncode)

metrics = None
if json_out.exists():
    metrics = json.loads(json_out.read_text(encoding="utf-8"))
    pprint(metrics)
else:
    print("metrics not generated")

run_dir_path = None
if metrics and metrics.get("run_dir"):
    run_dir_path = Path(metrics["run_dir"])
    print("run_dir=", run_dir_path)

daily_path = None
if run_dir_path:
    daily_path = run_dir_path / "daily.csv"
    if daily_path.exists():
        preview_lines = daily_path.read_text(encoding="utf-8").splitlines()[:5]
        print("daily.csv preview (first 5 lines):")
        for line in preview_lines:
            print(line)
    else:
        print("daily.csv not generated")
else:
    print("run_dir not found in metrics")


### インシデントフォルダへの成果物同期手順
- `runs/incident_{INCIDENT_ID}/metrics.json` を確認し、差分が固まったら `ops/incidents/{INCIDENT_ID}/replay_params.json` へコピーしてバージョン管理する。
- `runs/incident_{INCIDENT_ID}/daily.csv` や追加のダンプを見直し、必要に応じてインシデントディレクトリへ持ち帰る。
- この Notebook の考察メモは `ops/incidents/{INCIDENT_ID}/replay_notes.md` に整理し、次回レビュー時に参照できるよう同期する。
- 同期後は `git status` で変更を確認し、関連ファイルをコミットしてインシデント対応ログを最新化する。
